In [3]:
import gzip
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import torchvision as tv
import torch
import torch.nn as nn
import torch.nn.functional as F
from vit_pytorch import ViT

In [4]:
f = gzip.GzipFile('data_processed.npy.gz', "r")
data = np.load(f, allow_pickle=True)
f.close()
print(data, data.shape)
f = gzip.GzipFile('results_processed.npy.gz', "r")
labels = np.load(f, allow_pickle=True)
f.close()
print(labels, labels.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] (11785614, 768)
[1 1 1 ... 0 0 0] (11785614,)


In [5]:
#changing -1 to 2 cause crossentropy hates it or something
data = data[0:500000]
labels = labels[0:500000]
temp = []
for lab in labels:
    if lab == -1:
        temp.append(2)
    else:
        temp.append(lab)
        
labels = np.array(temp)

In [6]:
from dataset import Dataset

itr = int(len(data)*0.9)
train_data = data[0:itr]
train_labels = labels[0:itr]

test_data = data[itr:]
test_labels = labels[itr:]

params = {
    'batch_size':128,
    'shuffle':True,
    'num_workers':6
}

training_data_set = Dataset(train_labels,train_data)
training_generator = torch.utils.data.DataLoader(training_data_set, **params)


In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [8]:
def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader.dataset)
    model.train()

    for i, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())

In [9]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for data, target in data_loader:
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

In [24]:
eta = 0.001
epochs = 1

#model = ViT(image_size=768, patch_size=12, num_classes=3, channels=1,
#            dim=64, depth=6, heads=8, mlp_dim=128)
model = ViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=64, depth=6, heads=8, mlp_dim=128)

criterion = nn.CrossEntropyLoss()

AssertionError: embed_dim must be divisible by num_heads

In [19]:
#optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
optimizer = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

train_loss_history, test_loss_history = [], []
for epoch in range(1, epochs + 1):
    print('Epoch:', epoch)
    train_epoch(model, optimizer, training_generator, train_loss_history)
    evaluate(model, test_loader, test_loss_history)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')


Epoch: 1


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\huntm\anaconda3\envs\alt\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\huntm\anaconda3\envs\alt\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\huntm\anaconda3\envs\alt\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\huntm\Desktop\deep\chessAI\dataset.py", line 20, in __getitem__
    X = torch.load('data/' + ID + '.pt')
TypeError: can only concatenate str (not "numpy.int32") to str


In [22]:
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = nn.Transformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img, mask=None):
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=p, p2=p)
        x = self.patch_to_embedding(x)

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x, mask)

        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)